In [3]:
import numpy as np
import pandas as pd
import os 
from pathlib import Path
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.optimizers import Adam
from keras.layers import *
from keras.applications.inception_v3 import InceptionV3
from efficientnet.tfkeras import EfficientNetB0
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [4]:
image_dir = Path('/Users/geek/Downloads/geek/heart/0')

In [5]:
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: '0', filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

zerosimage_df = pd.concat([filepaths, labels], axis=1)

In [6]:
image_dir2 = Path('/Users/geek/Downloads/geek/heart/1')
filepaths2 = list(image_dir2.glob(r'**/*.png'))
labels2 = list(map(lambda x: '1', filepaths2))

filepaths2 = pd.Series(filepaths2, name='Filepath').astype(str)
labels2 = pd.Series(labels2, name='Label')

onesimage_df = pd.concat([filepaths2, labels2], axis=1)

In [7]:
image_df1 = [zerosimage_df,onesimage_df]
image_df = pd.concat(image_df1)


In [8]:
train_df,valid_df = train_test_split(image_df,train_size=0.8, shuffle = True , random_state=415)

In [9]:
train_df

,Filepath,Label
24,/Users/geek/Downloads/geek/heart/1/2311_2.png,1
70,/Users/geek/Downloads/geek/heart/1/613_1.png,1
112,/Users/geek/Downloads/geek/heart/1/1610_2.png,1
23,/Users/geek/Downloads/geek/heart/1/3518_6.png,1
605,/Users/geek/Downloads/geek/heart/1/149_1.png,1
...,...,...
120,/Users/geek/Downloads/geek/heart/0/15_3.png,0
464,/Users/geek/Downloads/geek/heart/1/1814_8.png,1
191,/Users/geek/Downloads/geek/heart/1/2812_1.png,1
338,/Users/geek/Downloads/geek/heart/0/1210_7.png,0


In [10]:
valid_df

,Filepath,Label
740,/Users/geek/Downloads/geek/heart/1/3615_2.png,1
397,/Users/geek/Downloads/geek/heart/1/1715_4.png,1
69,/Users/geek/Downloads/geek/heart/1/29_1.png,1
167,/Users/geek/Downloads/geek/heart/1/2719_4.png,1
184,/Users/geek/Downloads/geek/heart/0/1211_3.png,0
...,...,...
579,/Users/geek/Downloads/geek/heart/1/1814_7.png,1
247,/Users/geek/Downloads/geek/heart/1/1714_5.png,1
562,/Users/geek/Downloads/geek/heart/1/1211_1.png,1
778,/Users/geek/Downloads/geek/heart/1/289_7.png,1


In [11]:
idg = ImageDataGenerator(horizontal_flip = True)

In [14]:
train_generator = idg.flow_from_dataframe(train_df, x_col = "Filepath", y_col = "Label", target_size = (200,200), batch_size = 32)
valid_generator = idg.flow_from_dataframe(valid_df, x_col = "Filepath", y_col = "Label", target_size = (200,200), batch_size = 32)
test_generator = idg.flow_from_dataframe(test_df, x_col = "Filepath", y_col = "Label", target_size = (200,200), batch_size = 32)

Found 753 validated image filenames belonging to 2 classes.
Found 269 validated image filenames belonging to 2 classes.
Found 323 validated image filenames belonging to 2 classes.


In [13]:
train_df,test_df = train_test_split(train_df,train_size=0.7, shuffle = True , random_state=415)

In [15]:
train_df

,Filepath,Label
36,/Users/geek/Downloads/geek/heart/1/239_5.png,1
112,/Users/geek/Downloads/geek/heart/1/1610_2.png,1
100,/Users/geek/Downloads/geek/heart/1/3313_2.png,1
230,/Users/geek/Downloads/geek/heart/0/1311_1.png,0
434,/Users/geek/Downloads/geek/heart/1/2718_5.png,1
...,...,...
167,/Users/geek/Downloads/geek/heart/0/815_5.png,0
247,/Users/geek/Downloads/geek/heart/0/718_1.png,0
436,/Users/geek/Downloads/geek/heart/1/139_2.png,1
664,/Users/geek/Downloads/geek/heart/1/329_6.png,1


In [16]:
test_df

,Filepath,Label
346,/Users/geek/Downloads/geek/heart/1/612_3.png,1
773,/Users/geek/Downloads/geek/heart/1/613_8.png,1
9,/Users/geek/Downloads/geek/heart/0/14_2.png,0
282,/Users/geek/Downloads/geek/heart/0/1017_2.png,0
409,/Users/geek/Downloads/geek/heart/1/612_6.png,1
...,...,...
820,/Users/geek/Downloads/geek/heart/1/2912_8.png,1
506,/Users/geek/Downloads/geek/heart/1/1816_7.png,1
707,/Users/geek/Downloads/geek/heart/1/214_2.png,1
146,/Users/geek/Downloads/geek/heart/0/819_4.png,0


In [17]:

model = Sequential()
model.add(InceptionV3(weights = "imagenet" , include_top = False, pooling = "avg"))
model.add(Dense(2, activation = "relu"))
adam = Adam(lr=0.001)
model.compile(metrics = ["acc"], loss = "binary_crossentropy", optimizer = adam)
model.fit(train_generator ,validation_data=valid_generator, epochs = 120)
results1 = model.predict(test_generator)

2022-12-28 18:48:32.253759: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-28 18:48:32.255098: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Epoch 1/120


/opt/homebrew/Caskroom/miniforge/base/envs/geek/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-12-28 18:48:34.878358: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-28 18:48:36.601441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 2.3192 - acc: 0.8141

2022-12-28 18:49:07.036545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 35s 889ms/step - loss: 2.3192 - acc: 0.8141 - val_loss: 5.9614 - val_acc: 0.7398
Epoch 2/120
24/24 [==============================] - 16s 684ms/step - loss: 0.7778 - acc: 0.9402 - val_loss: 3.9911 - val_acc: 0.7398
Epoch 3/120
24/24 [==============================] - 14s 589ms/step - loss: 0.4401 - acc: 0.9402 - val_loss: 3.9341 - val_acc: 0.7435
Epoch 4/120
24/24 [==============================] - 14s 581ms/step - loss: 0.1191 - acc: 0.9867 - val_loss: 3.3665 - val_acc: 0.7770
Epoch 5/120
24/24 [==============================] - 14s 568ms/step - loss: 0.2848 - acc: 0.9681 - val_loss: 3.0612 - val_acc: 0.7993
Epoch 6/120
24/24 [==============================] - 14s 577ms/step - loss: 0.2408 - acc: 0.9788 - val_loss: 0.4467 - val_acc: 0.9703
Epoch 7/120
24/24 [==============================] - 14s 574ms/step - loss: 0.6403 - acc: 0.9363 - val_loss: 2.3648 - val_acc: 0.8550
Epoch 8/120
24/24 [==============================] - 14s 582ms/step - loss

Epoch 62/120
24/24 [==============================] - 14s 564ms/step - loss: 0.3725 - acc: 0.9270 - val_loss: 0.5476 - val_acc: 0.7732
Epoch 63/120
24/24 [==============================] - 13s 556ms/step - loss: 0.4922 - acc: 0.8340 - val_loss: 0.5355 - val_acc: 0.8327
Epoch 64/120
24/24 [==============================] - 15s 606ms/step - loss: 0.3305 - acc: 0.9230 - val_loss: 0.4455 - val_acc: 0.9368
Epoch 65/120
24/24 [==============================] - 13s 556ms/step - loss: 0.3291 - acc: 0.9336 - val_loss: 0.4917 - val_acc: 0.8178
Epoch 66/120
24/24 [==============================] - 13s 556ms/step - loss: 0.1694 - acc: 0.9535 - val_loss: 0.4004 - val_acc: 0.9517
Epoch 67/120
24/24 [==============================] - 13s 558ms/step - loss: 0.2785 - acc: 0.9110 - val_loss: 0.4442 - val_acc: 0.8141
Epoch 68/120
24/24 [==============================] - 13s 551ms/step - loss: 0.3725 - acc: 0.8260 - val_loss: 0.4177 - val_acc: 0.7584
Epoch 69/120
24/24 [==============================] - 1

2022-12-28 19:16:00.313136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 5s 314ms/step


In [18]:
results = model.evaluate(test_generator, verbose = 0)
print("  test loss: {:.5f}".format(results[0]))
print("test accuracy: {:.2f}%".format(results[1]*100))

  test loss: 0.21241
test accuracy: 91.95%
